# Models

### Testing models

Using the two following models:

https://huggingface.co/cardiffnlp/twitter-roberta-base-hate-latest

https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dtale
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline
import torch

from master_thesis.config import PROCESSED_DATA_DIR, load_dataframe_from_pickle

In [ ]:
annotated_and_targeted_hatespeech_df = load_dataframe_from_pickle(
    PROCESSED_DATA_DIR / "annotated_and_targeted_hatespeech.pkl"
)

display(annotated_and_targeted_hatespeech_df.head(10))

In [ ]:
test_text_0 = annotated_and_targeted_hatespeech_df['text'][2]
test_text_1 = annotated_and_targeted_hatespeech_df['text'][6]

In [ ]:
device = 0 if torch.cuda.is_available() else -1


In [ ]:

pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-hate-latest")


In [ ]:
result = pipe(test_text_0)
print(result)

In [ ]:
result = pipe(test_text_1)
print(result)

In [ ]:
model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
classifier = pipeline("sentiment-analysis", model=model_name, framework="pt", device=device)


In [ ]:
result = classifier(test_text_0)
print(result)

In [ ]:
result = classifier(test_text_1)
print(result)

### Using LIME


In [ ]:
from lime.lime_text import LimeTextExplainer

In [ ]:
class_names = ['not_hate', 'hate']

# Model 1: CardiffNLP
pipe_cardiff = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-hate-latest", return_all_scores=True)

# Model 2: Facebook Dynabench
pipe_fb = pipeline("text-classification", model="facebook/roberta-hate-speech-dynabench-r4-target", return_all_scores=True)


In [ ]:
def predict_cardiff(texts):
    return np.array([[score['score'] for score in pipe_cardiff(text)[0]] for text in texts])

def predict_fb(texts):
    return np.array([[score['score'] for score in pipe_fb(text)[0]] for text in texts])

In [ ]:
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
explanation_cardiff = explainer.explain_instance(test_text_0, predict_cardiff, num_features=10)
explanation_cardiff.show_in_notebook(text=True)


In [ ]:
explanation_cardiff = explainer.explain_instance(test_text_1, predict_cardiff, num_features=10)
explanation_cardiff.show_in_notebook(text=True)

In [ ]:
explanation_fb = explainer.explain_instance(test_text_0, predict_fb, num_features=10)
explanation_fb.show_in_notebook(text=True)

In [ ]:
explanation_fb = explainer.explain_instance(test_text_1, predict_fb, num_features=10)
explanation_fb.show_in_notebook(text=True)

### SHAP

In [ ]:
import shap

In [ ]:
text_list_0 = [test_text_0]
text_list_1 = [test_text_1]

In [ ]:
# Model 1: CardiffNLP
pipe_cardiff = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-hate-latest", return_all_scores=True)

# Model 2: Facebook Dynabench
pipe_fb = pipeline("text-classification", model="facebook/roberta-hate-speech-dynabench-r4-target", return_all_scores=True)


In [ ]:
from transformers import AutoTokenizer
tokenizer_cardiff = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
tokenizer_fb = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")

In [ ]:
def predict_cardiff(texts):
    return np.array([[label['score'] for label in pipe_cardiff(text)[0]] for text in texts])

def predict_fb(texts):
    return np.array([[label['score'] for label in pipe_fb(text)[0]] for text in texts])


In [ ]:
explainer_cardiff = shap.Explainer(
    predict_cardiff,
    masker=shap.maskers.Text(tokenizer_cardiff),
    output_names=class_names
)

explainer_fb = shap.Explainer(
    predict_fb,
    masker=shap.maskers.Text(tokenizer_fb),
    output_names=class_names
)


In [ ]:
shap_values_cardiff = explainer_cardiff(text_list_0)
shap_values_fb = explainer_fb(text_list_0)

In [ ]:
shap.plots.text(shap_values_cardiff[0])

In [ ]:
shap.plots.text(shap_values_fb[0])

In [ ]:
shap_values_cardiff = explainer_cardiff(text_list_1)
shap_values_fb = explainer_fb(text_list_1)

In [ ]:
shap.plots.text(shap_values_cardiff[0])

In [ ]:
shap.plots.text(shap_values_fb[0])

In [ ]:
# Compute SHAP values for a single instance and a specific class
shap_values_cardiff = explainer_cardiff(text_list_0)
shap_values_fb = explainer_fb(text_list_0)

# Select SHAP values for the "hate" class (index 1)
shap_values_cardiff_hate = shap_values_cardiff[0][:, 1]
shap_values_fb_hate = shap_values_fb[0][:, 1]

# Generate waterfall plots for the "hate" class
print("Waterfall plot for CardiffNLP model:")
shap.plots.waterfall(shap_values_cardiff_hate)
print("Waterfall plot for Facebook Dynabench model:")
shap.plots.waterfall(shap_values_fb_hate)

In [ ]:
# Compute SHAP values for a single instance and a specific class
shap_values_cardiff = explainer_cardiff(text_list_1)
shap_values_fb = explainer_fb(text_list_1)

# Select SHAP values for the "hate" class (index 1)
shap_values_cardiff_hate = shap_values_cardiff[0][:, 1]
shap_values_fb_hate = shap_values_fb[0][:, 1]

# Generate waterfall plots for the "hate" class
print("Waterfall plot for CardiffNLP model:")
shap.plots.waterfall(shap_values_cardiff_hate)
print("Waterfall plot for Facebook Dynabench model:")
shap.plots.waterfall(shap_values_fb_hate)

### Applying to dataframe

In [1]:
from lime.lime_text import LimeTextExplainer
import shap
import numpy as np
from transformers import pipeline, AutoTokenizer
import random
import torch


/home/takosaga/miniconda3/envs/master_thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-20 21:08:57.859215: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-20 21:08:57.884173: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-20 21:08:57.884203: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-20 21:08:57.884225: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory fo

In [2]:

def set_seed(seed_value):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        # Optional: for determinism with CuDNN
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False



In [3]:
seed_value = 42
set_seed(seed_value)

class_names = ['not_hate', 'hate']

# Pipelines
pipe_cardiff = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-hate-latest")
pipe_fb = pipeline("text-classification", model="facebook/roberta-hate-speech-dynabench-r4-target")

# Tokenizers
tokenizer_cardiff = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
tokenizer_fb = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")

# Prediction functions for LIME (no [0] indexing)
def predict_cardiff(texts):
    # Ensure input is list of strings
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
    return np.array([
        [res['score'] for res in pipe_cardiff(text, top_k=5)]
        for text in texts
    ])

def predict_fb(texts):
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
    return np.array([
        [res['score'] for res in pipe_fb(text, top_k=5)]
        for text in texts
    ])

# LIME & SHAP explainers
explainer_lime = LimeTextExplainer(class_names=class_names)
explainer_shap_cardiff = shap.Explainer(predict_cardiff, masker=shap.maskers.Text(tokenizer_cardiff), output_names=class_names)
explainer_shap_fb = shap.Explainer(predict_fb, masker=shap.maskers.Text(tokenizer_fb), output_names=class_names)


Device set to use cpu
Device set to use cpu


In [4]:
import pandas as pd
data = {'text': ["nawt yall niggers ignoring me",
                 "and this is why i end up with nigger trainee doctors who can not speak properly lack basic knowledge of biology it truly scary if the public only knew",
                 "this is a neutral sentence"]}
annotated_and_targeted_hatespeech_df = pd.DataFrame(data)

The following was individually

In [5]:
lime_cardiff_explanations = []
lime_fb_explanations = []
shap_cardiff_explanations = []
shap_fb_explanations = []


In [6]:

# ---- LIME: Cardiff ----
for text in annotated_and_targeted_hatespeech_df["text"]:
    lime_cardiff = explainer_lime.explain_instance(text, predict_cardiff, num_features=5)
    lime_cardiff_keywords = [f"{word}: {weight:.2f}" for word, weight in lime_cardiff.as_list()]
    lime_cardiff_explanations.append(", ".join(lime_cardiff_keywords))


KeyboardInterrupt: 

In [7]:
annotated_and_targeted_hatespeech_df["lime_cardiff"] = lime_cardiff_explanations

In [8]:

# ---- LIME: Facebook ----
for text in annotated_and_targeted_hatespeech_df["text"]:
    lime_fb = explainer_lime.explain_instance(text, predict_fb, num_features=5)
    lime_fb_keywords = [f"{word}: {weight:.2f}" for word, weight in lime_fb.as_list()]
    lime_fb_explanations.append(", ".join(lime_fb_keywords))


In [9]:
annotated_and_targeted_hatespeech_df["lime_fb"] = lime_fb_explanations

In [10]:

# ---- SHAP: Cardiff ----
for text in annotated_and_targeted_hatespeech_df["text"]:
    shap_vals_cardiff = explainer_shap_cardiff([text])
    shap_cardiff_words = [
        f"{feature}: {value:.2f}"
        for feature, value in zip(shap_vals_cardiff[0].data, shap_vals_cardiff[0].values[:, 1])
        if abs(value) > 0.01
    ]
    shap_cardiff_explanations.append(", ".join(shap_cardiff_words))


PartitionExplainer explainer: 2it [00:20, 20.15s/it]               


In [11]:
annotated_and_targeted_hatespeech_df["shap_cardiff"] = shap_cardiff_explanations

In [7]:

# ---- SHAP: Facebook ----
for text in annotated_and_targeted_hatespeech_df["text"]:
    shap_vals_fb = explainer_shap_fb([text])
    shap_fb_words = [
        f"{feature}: {value:.2f}"
        for feature, value in zip(shap_vals_fb[0].data, shap_vals_fb[0].values[:, 1])
        if abs(value) > 0.01
    ]
    shap_fb_explanations.append(", ".join(shap_fb_words))


KeyboardInterrupt: 

In [8]:
annotated_and_targeted_hatespeech_df["shap_fb"] = shap_fb_explanations


ValueError: Length of values (0) does not match length of index (3)

In [25]:
annotated_and_targeted_hatespeech_df

,text,lime_cardiff,lime_fb,shap_cardiff,shap_fb
0,nawt yall niggers ignoring me,"nawt: -0.09, niggers: 0.08, me: 0.07, yall: 0....","nawt: -0.02, ignoring: -0.02, yall: -0.01, me:...","n: 0.01, nig: 0.04, gers : -0.03, ignoring : 0.02","n: 0.05, aw: -0.02, t : 0.04, y: -0.01, all : ..."
1,and this is why i end up with nigger trainee d...,"with: -0.01, nigger: 0.01, trainee: -0.00, i: ...","nigger: -0.02, scary: 0.00, trainee: 0.00, and...",,"and : -0.03, end : -0.01, n: -0.03, igger : -0..."
2,this is a neutral sentence,"a: -0.00, is: -0.00, this: -0.00, sentence: -0...","neutral: -0.00, this: -0.00, sentence: 0.00, a...",,"this : -0.10, is : 0.04, a : 0.04, sentence: -..."


Following is for each text

In [9]:
class_names = ['not_hate', 'hate']
explainer_lime = LimeTextExplainer(class_names=class_names)

# Simplified LIME explainers
def lime_explain(text, predictor, num_features=4):
    exp = explainer_lime.explain_instance(text, predictor, num_features=num_features)
    return ", ".join([f"{word}: {weight:.2f}" for word, weight in exp.as_list()])

# Predictor wrappers
def predict_cardiff(texts):
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
    outputs = pipe_cardiff(texts, top_k=2)
    return np.array([
        [label['score'] for label in sorted(res, key=lambda x: x['label'])]
        for res in outputs
    ])

def predict_fb(texts):
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
    outputs = pipe_fb(texts, top_k=2)
    return np.array([
        [label['score'] for label in sorted(res, key=lambda x: x['label'])]
        for res in outputs
    ])

In [10]:
def get_shap_values(text, explainer):
    shap_vals = explainer([text])
    return ", ".join([
        f"{feature}: {value:.2f}"
        for feature, value in zip(shap_vals[0].data, shap_vals[0].values)
        if abs(value) > 0.01
    ])

In [11]:
results = []

In [ ]:
for text in annotated_and_targeted_hatespeech_df["text"]:
    try:
        # Get predictions
        pred_cardiff = pipe_cardiff(text)[0]
        pred_fb = pipe_fb(text)[0]

        # LIME
        lime_exp_cardiff = lime_explain(text, predict_cardiff)
        lime_exp_fb = lime_explain(text, predict_fb)

        # SHAP
        shap_vals_cardiff = explainer_shap_cardiff([text])
        shap_exp_cardiff = ", ".join([
            f"{feature}: {value:.3f}"
            for feature, value in zip(shap_vals_cardiff[0].data, shap_vals_cardiff[0].values[:, 1])  # Select class index 1 (hate)
            if abs(value) > 0.001
        ])

        shap_vals_fb = explainer_shap_fb([text])
        shap_exp_fb = ", ".join([
            f"{feature}: {value:.3f}"
            for feature, value in zip(shap_vals_fb[0].data, shap_vals_fb[0].values[:, 1])  # Select class index 1 (hate)
            if abs(value) > 0.001
        ])

        # Store results
        results.append({
            "text": text,
            "error": None,
            "cardiff_label": pred_cardiff["label"],
            "cardiff_score": pred_cardiff["score"],
            "fb_label": pred_fb["label"],
            "fb_score": pred_fb["score"],
            "lime_cardiff": lime_exp_cardiff,
            "lime_fb": lime_exp_fb,
            "shap_cardiff": shap_exp_cardiff,
            "shap_fb": shap_exp_fb
        })
    except Exception as e:
        results.append({
            "text": text,
            "error": f"Caught an exception: {e}",
            "cardiff_label": None,
            "cardiff_score": None,
            "fb_label": None,
            "fb_score": None,
            "lime_cardiff": None,
            "lime_fb": None,
            "shap_cardiff": None,
            "shap_fb": None
        })
        
    

PartitionExplainer explainer: 2it [00:19, 19.54s/it]               
PartitionExplainer explainer: 2it [00:18, 18.31s/it]               


In [14]:
final_df = pd.DataFrame(results)

# Preview
final_df.head()

,text,cardiff_label,cardiff_score,fb_label,fb_score,lime_cardiff,lime_fb,shap_cardiff,shap_fb
0,nawt yall niggers ignoring me,HATE,0.935527,hate,0.999480,"niggers: -0.91, nawt: -0.09, me: 0.07, yall: 0.07","niggers: -0.98, ignoring: -0.03, nawt: -0.02, ...","n: 0.010, aw: 0.006, t : 0.006, y: 0.003, all ...","n: 0.047, aw: -0.016, t : 0.040, y: -0.010, al..."
1,and this is why i end up with nigger trainee d...,HATE,0.993975,hate,0.995142,"nigger: -0.98, with: -0.01, i: 0.00, doctors: ...","nigger: -0.97, scary: -0.00, speak: -0.00, who...","and : -0.002, this : -0.001, with : -0.001, n:...","and : -0.028, is : 0.005, why : -0.007, i : -0..."
2,this is a neutral sentence,NOT-HATE,0.999157,nothate,0.999490,"a: 0.00, is: 0.00, this: 0.00, sentence: 0.00","neutral: 0.00, this: 0.00, sentence: -0.00, a:...",,"this : -0.100, is : 0.036, a : 0.040, neutral ..."


### Setting Up for Collab

In [ ]:
pip install lime shap hf_xet

In [ ]:
from lime.lime_text import LimeTextExplainer
import shap
import numpy as np
import pandas as pd
from transformers import pipeline, AutoTokenizer
import random
import torch
from typing import List, Union


def set_seed(seed_value):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        # Optional: for determinism with CuDNN
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False


# Set seed for reproducibility
set_seed(42)

# Constants
CLASS_NAMES = ['not_hate', 'hate']
BATCH_SIZE = 32  # Adjust based on your GPU memory

# Load models and tokenizers once
pipe_cardiff = pipeline("text-classification", 
                       model="cardiffnlp/twitter-roberta-base-hate-latest", 
                       device=0 if torch.cuda.is_available() else -1,
                       batch_size=BATCH_SIZE)
pipe_fb = pipeline("text-classification", 
                  model="facebook/roberta-hate-speech-dynabench-r4-target", 
                  device=0 if torch.cuda.is_available() else -1,
                  batch_size=BATCH_SIZE)

tokenizer_cardiff = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")
tokenizer_fb = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")


# Create dataset class for more efficient processing
class HateSpeechDataset:
    def __init__(self, texts):
        self.texts = texts
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx]


# Unified prediction function for both models
def batch_predict(texts: Union[List[str], np.ndarray], pipeline_fn):
    """Run predictions in batches for efficiency"""
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
        
    dataset = HateSpeechDataset(texts)
    
    # Process in batches
    all_outputs = []
    for i in range(0, len(dataset), BATCH_SIZE):
        batch_texts = [dataset[j] for j in range(i, min(i + BATCH_SIZE, len(dataset)))]
        outputs = pipeline_fn(batch_texts, top_k=2)
        all_outputs.extend(outputs)
    
    # Convert to numpy array with consistent order
    return np.array([
        [label['score'] for label in sorted(res, key=lambda x: x['label'])]
        for res in all_outputs
    ])


# Model-specific prediction functions
def predict_cardiff(texts):
    return batch_predict(texts, pipe_cardiff)


def predict_fb(texts):
    return batch_predict(texts, pipe_fb)


# Initialize explainers
explainer_lime = LimeTextExplainer(class_names=CLASS_NAMES)
explainer_shap_cardiff = shap.Explainer(predict_cardiff, 
                                       masker=shap.maskers.Text(tokenizer_cardiff), 
                                       output_names=CLASS_NAMES)
explainer_shap_fb = shap.Explainer(predict_fb, 
                                  masker=shap.maskers.Text(tokenizer_fb), 
                                  output_names=CLASS_NAMES)


# Helper function for LIME explanations
def lime_explain(text, predictor, num_features=4):
    exp = explainer_lime.explain_instance(text, predictor, num_features=num_features)
    return ", ".join([f"{word}: {weight:.2f}" for word, weight in exp.as_list()])


# Helper function for SHAP explanations
def get_shap_values(text, explainer, class_idx=1):
    shap_vals = explainer([text])
    return ", ".join([
        f"{feature}: {value:.3f}"
        for feature, value in zip(shap_vals[0].data, shap_vals[0].values[:, class_idx])
        if abs(value) > 0.001
    ])


# Sample data
data = {
    'text': [
        "nawt yall niggers ignoring me",
        "and this is why i end up with nigger trainee doctors who can not speak properly lack basic knowledge of biology it truly scary if the public only knew",
        "this is a neutral sentence"
    ]
}
annotated_and_targeted_hatespeech_df = pd.DataFrame(data)


# Process all texts in one batch for efficiency
all_texts = annotated_and_targeted_hatespeech_df["text"].tolist()

# Get predictions in batch
cardiff_preds = pipe_cardiff(all_texts) 
fb_preds = pipe_fb(all_texts)

# Process results
results = []
for i, text in enumerate(all_texts):
    try:
        # Get predictions from the batch results
        pred_cardiff = cardiff_preds[i]
        pred_fb = fb_preds[i]

        # LIME explanations
        lime_exp_cardiff = lime_explain(text, predict_cardiff)
        lime_exp_fb = lime_explain(text, predict_fb)

        # SHAP explanations
        shap_exp_cardiff = get_shap_values(text, explainer_shap_cardiff)
        shap_exp_fb = get_shap_values(text, explainer_shap_fb)

        # Store results
        results.append({
            "text": text,
            "error": None,
            "cardiff_label": pred_cardiff["label"],
            "cardiff_score": pred_cardiff["score"],
            "fb_label": pred_fb["label"],
            "fb_score": pred_fb["score"],
            "lime_cardiff": lime_exp_cardiff,
            "lime_fb": lime_exp_fb,
            "shap_cardiff": shap_exp_cardiff,
            "shap_fb": shap_exp_fb
        })
    except Exception as e:
        results.append({
            "text": text,
            "error": f"Caught an exception: {e}",
            "cardiff_label": None,
            "cardiff_score": None,
            "fb_label": None,
            "fb_score": None,
            "lime_cardiff": None,
            "lime_fb": None,
            "shap_cardiff": None,
            "shap_fb": None
        })

final_df = pd.DataFrame(results)


# Print the results
final_df.head()

/home/takosaga/miniconda3/envs/master_thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-20 22:44:34.369543: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-20 22:44:34.398433: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-20 22:44:34.398466: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-20 22:44:34.398493: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory fo

,text,error,cardiff_label,cardiff_score,fb_label,fb_score,lime_cardiff,lime_fb,shap_cardiff,shap_fb
0,nawt yall niggers ignoring me,None,HATE,0.935527,hate,0.999480,"niggers: -0.91, nawt: -0.10, me: 0.07, yall: 0.07","niggers: -0.98, ignoring: -0.03, nawt: -0.02, ...","n: 0.010, aw: 0.006, t : 0.007, y: 0.003, all ...","n: 0.030, aw: -0.062, t : -0.040, y: 0.129, al..."
1,and this is why i end up with nigger trainee d...,None,HATE,0.993975,hate,0.995142,"nigger: -0.98, with: -0.01, i: 0.00, doctors: ...","nigger: -0.97, who: -0.00, speak: -0.00, scary...","and : 0.003, this : 0.001, n: -0.007, igger : ...","and : 0.027, is : -0.003, why : 0.009, i : 0.0..."
2,this is a neutral sentence,None,NOT-HATE,0.999157,nothate,0.999490,"a: 0.00, is: 0.00, this: 0.00, sentence: 0.00","neutral: 0.00, this: 0.00, sentence: -0.00, is...",,"this : 0.100, is : -0.036, a : -0.040, neutral..."


In [ ]:
# Saving to drive
from google.colab import drive
drive.mount('/content/drive')

# Save to your Google Drive
final_df.to_csv('/content/drive/MyDrive/hate_speech_analysis_results.csv', index=False)
final_df.to_excel('/content/drive/MyDrive/hate_speech_analysis_results.xlsx', index=False)
final_df.to_pickle('/content/drive/MyDrive/hate_speech_analysis_results.pkl')


In [16]:
def process_dataframe(df, text_column="text"):
    """Process an entire dataframe with efficient batching"""
    all_texts = df[text_column].tolist()
    
    # Get predictions in batch
    cardiff_preds = pipe_cardiff(all_texts)
    fb_preds = pipe_fb(all_texts)
    
    results = []
    for i, text in enumerate(all_texts):
        try:
            pred_cardiff = cardiff_preds[i]
            pred_fb = fb_preds[i]
            
            # Calculate explanations for all examples
            lime_exp_cardiff = lime_explain(text, predict_cardiff)
            lime_exp_fb = lime_explain(text, predict_fb)
            shap_exp_cardiff = get_shap_values(text, explainer_shap_cardiff)
            shap_exp_fb = get_shap_values(text, explainer_shap_fb)
            
            results.append({
                "text": text,
                "error": None,
                "cardiff_label": pred_cardiff["label"],
                "cardiff_score": pred_cardiff["score"],
                "fb_label": pred_fb["label"],
                "fb_score": pred_fb["score"],
                "lime_cardiff": lime_exp_cardiff,
                "lime_fb": lime_exp_fb,
                "shap_cardiff": shap_exp_cardiff,
                "shap_fb": shap_exp_fb
            })
        except Exception as e:
            results.append({
                "text": text,
                "error": f"Caught an exception: {e}",
                "cardiff_label": None,
                "cardiff_score": None,
                "fb_label": None,
                "fb_score": None,
                "lime_cardiff": None,
                "lime_fb": None,
                "shap_cardiff": None,
                "shap_fb": None
            })
    
    return pd.DataFrame(results)